In [76]:
import pandas as pd

In [77]:
df = pd.read_csv(r"C:\Users\aryan\OneDrive\Desktop\Capstone Project\Data Preprocessing New\data_recommendation_v2.csv")

In [78]:
df.sample(1)

,Unnamed: 0,Property Name,Sector,Price,Built Up Area,Bedroom,Bathroom,Balcony,Floor Num,Total Floor,Property Age,URL,Furnishing,Covered_Parking,Open_Parking,Total Parking,Rating,Power Backup,Facing,Overlooking,Extra_Rooms,NearbyPlaces
5226,5306,DLF Regal Gardens,Sector 90,2.07,1608,3,3,3,4,29,2,https://www.99acres.com/3-bhk-bedroom-apartmen...,1,1,1,2,4.02,1,North-East,Club,1,"['""SS Omnia, Sector 86', 'Numberdar market, IM..."


In [79]:
df.drop(columns= 'Unnamed: 0', inplace= True)

In [80]:
df['NearbyPlaces'].iloc[0]

"['Millennium City Centre Gurugram', 'Town Square 2', 'Manesar Road', 'Sh. S. N. Sidheshwar Sr. Sec. Public', 'Singhania University']"

In [86]:
df.isnull().sum()

Property Name        0
Sector               0
Price                0
Built Up Area        0
Bedroom              0
Bathroom             0
Balcony              0
Floor Num            0
Total Floor          0
Property Age         0
URL                  0
Furnishing           0
Covered_Parking      0
Open_Parking         0
Total Parking        0
Rating               0
Power Backup         0
Facing               0
Overlooking          0
Extra_Rooms          0
NearbyPlaces       159
dtype: int64

In [87]:
df = df[~(df['NearbyPlaces'].isnull())]

In [88]:
df.shape

(9441, 21)

In [89]:
import ast
# Use ast.literal_eval to safely convert the string back into a list
df['NearbyPlaces'] = df['NearbyPlaces'].apply(ast.literal_eval)

In [90]:
df['NearbyPlacesStr'] = df['NearbyPlaces'].apply(' '.join)

In [143]:
df['NearbyPlacesStr'].iloc[0]

'Millennium City Centre Gurugram Town Square 2 Manesar Road Sh. S. N. Sidheshwar Sr. Sec. Public Singhania University'

In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [145]:
df['feature_text'] = df['Sector'].str.replace(' ', '') + ' ' + df['NearbyPlaces'].apply(lambda x: ' '.join(x).replace(' ', ''))

In [146]:
df.iloc[0]['feature_text']

'Sector81 MillenniumCityCentreGurugramTownSquare2ManesarRoadSh.S.N.SidheshwarSr.Sec.PublicSinghaniaUniversity'

In [147]:
tfidf = TfidfVectorizer(stop_words= 'english')

In [148]:
tfidf_matrix = tfidf.fit_transform(df['feature_text'])

In [149]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [153]:
def recommend_properties(property_name, cosine_sim_matrix= cosine_sim, df= df):
    '''Find top 5 most similar properties to a given property'''
    try:
        idx = df[df['Property Name'] == property_name].index[0]
    except IndexError:
        return f"Property '{property_name}' not found in the dataset."
    
    
    # Get the pairwise similarity scores for this property
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    
    # Sort the properties based on the similarity scores
    sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse= True)
    
    # Get the scores of the 5 most similar properties (the first one is the property itself)
    # 0 is that property itself
    sim_scores = sim_scores[1:500]
    
    property_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] for i in sim_scores]
    
    recommendations_df = pd.DataFrame({
        'PropertyName': df['Property Name'].iloc[property_indices],
        'SimilarityScore': similarity_scores
    })
    
    # 2. Now, drop the duplicates based on the property name
    unique_recommendations = recommendations_df.drop_duplicates(subset=['PropertyName'])
    
    final_recommendations = unique_recommendations[unique_recommendations['PropertyName'] != property_name]
    
    # 3. Return the top 5 from this unique list
    return final_recommendations.head(5)

In [154]:
df.sample(1)['Property Name'].iloc[0]

'Parsvnath Green Ville'

In [156]:
recommend_properties('Parsvnath Green Ville')

,PropertyName,SimilarityScore
16,M3M Heights,1.000000
565,Emaar MGF Emerald Floors Premier,0.281845
1987,"Sector 65, Gurgaon",0.240551
79,Emaar Emerald Hills,0.233662
2988,Emmar Hills,0.221157
